### 제주도 도로 교통량 예측 AI 경진대회

#### CatBoost + XGBoost + LGBM Ensemble 모델 (Private 3.08359) (게더타운주민들)

- 어린이보호구역 : https://www.data.go.kr/data/15012891/standard.do
- 무인교통단속카메라 : https://www.data.go.kr/data/15028200/standard.do
- 전국초중등학교기본정보 : https://www.data.go.kr/data/15107734/standard.do
- 제주도 주차장 : https://www.data.go.kr/data/15012896/standard.do

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import gc

from sklearn.preprocessing import LabelEncoder
from haversine import haversine
from sklearn.cluster import KMeans

import math

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor, Pool
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

### Data

In [2]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'data/jeju/{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [3]:
csv_to_parquet('data/jeju/train.csv', 'train')
csv_to_parquet('data/jeju/test.csv', 'test')

train Done.
test Done.


In [4]:
train = pd.read_parquet('data/jeju/train.parquet')
test = pd.read_parquet('data/jeju/test.parquet')

# 불필요 컬럼 삭제
train.drop(['vehicle_restricted', 'id', 'height_restricted'], axis = 1, inplace = True)
test.drop(['vehicle_restricted', 'id', 'height_restricted'], axis = 1, inplace = True)

### Feature Engineering

#### 1. 도로 주변 시설 및 구역 수(train 기준)
- 공공 데이터 포털에서 2022년 12월 이전 아래 5가지 표준 데이터를 사용
    - 무인교통단속카메라
    - 전국초중등학교기본정보
    - 어린이보호구역
    - 제주시 주차장 정보
    - 서귀포시 주차장 정보
- train의 start_node, end_node의 위경도 좌표의 unique 값만을 활용

In [5]:
# train의 start_node, end_node의 위경도 좌표의 unique 값
gps_comb = train[['start_latitude', 'start_longitude', 'end_latitude', 'end_longitude']].drop_duplicates()

# 무인교통단속카메라 - 데이터 내려감
# cctv = pd.read_csv('경찰청_제주특별자치도경찰청_무인교통단속카메라_20220616.csv', encoding = 'cp949')
# cctv = cctv.iloc[:, 3:-7].drop(['소재지도로명주소', '소재지지번주소'], axis = 1)

# 초중등학교
school = pd.read_csv('data/jeju/한국교육학술정보원_초중등학교기본정보_20221213.csv', encoding = 'cp949')
school = school[(school['데이터기준일자'] <= '2022-12-01') & (school['시도교육청명'].str.contains('제주'))]

# 어린이 보호 구역
child = pd.read_csv('data/jeju/제주특별자치도_어린이보호구역_20221201.csv', encoding = 'cp949')

# 제주시 주차장
parking1 = pd.read_csv('data/jeju/제주특별자치도_제주시_주차장정보_20210818_1630391997093_77385.csv', encoding = 'cp949')
parking1.dropna(subset = ['위도', '경도'], inplace = True)

# 서귀포시 주차장
parking2 = pd.read_csv('data/jeju/제주특별자치도_서귀포시_주차장정보_20221215.csv', encoding = 'cp949')

In [6]:
# 직선과 점 사이의 거리 방정식 활용
def cal_dist(x1, y1, x2, y2, a, b):
    area = abs((x1 - a) * (y2 - b) - (y1 - b) * (x2 - a))
    AB = ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5
    distance = area / AB
    return distance

In [7]:
# train 데이터의 도로와 각 시설 및 구역의 위경도 좌표의 거리(위경도 좌표상 거리)가 0.0005이내일 경우 count +
def get_node_cnt(gps_values, infra_values):
    cnt = []
    for y1, x1, y2, x2 in gps_values.values:
        i = 0
        for a, b in infra_values.values:
            dist = cal_dist(x1, y1, x2, y2, a, b)
            if dist < 0.0005:
                i += 1
            else:
                pass
        cnt.append(i)
    return cnt

In [8]:
school.head()

,시도교육청코드,시도교육청명,교육지원청코드,교육지원청명,시도코드,시도명,시군구코드,시군구명,학교명,설립형태 구분명,...,교원 수,특수학급 수,소재지도로명주소,소재지지번주소,홈페이지주소,전화번호,팩스번호,남녀공학구분명,공시차수,데이터기준일자
10846,9290000,제주특별자치도교육청,9299000,제주특별자치도서귀포시교육지원청,50,제주특별자치도,130,서귀포시,대정고등학교,공립,...,43,1,제주특별자치도 서귀포시 대정읍 일주서로2498번길 21,NaN,http://daejeong.jje.hs.kr,064-730-0100,064-794-2070,남,202205,2022-05-31
10847,9290000,제주특별자치도교육청,9299000,제주특별자치도서귀포시교육지원청,50,제주특별자치도,130,서귀포시,서귀포온성학교,공립,...,45,2,제주특별자치도 서귀포시 516로 447-10,NaN,http://onseong.jje.sc.kr,064-760-8000,064-767-8014,남녀공학,202205,2022-05-31
10848,9290000,제주특별자치도교육청,9299000,제주특별자치도서귀포시교육지원청,50,제주특별자치도,130,서귀포시,토산초등학교,공립,...,10,0,제주특별자치도 서귀포시 표선면 토산중앙로 68-9,NaN,http://tosan.jje.es.kr,064-780-9200,064-780-9299,남녀공학,202205,2022-05-31
10849,9290000,제주특별자치도교육청,9299000,제주특별자치도서귀포시교육지원청,50,제주특별자치도,130,서귀포시,남원초등학교,공립,...,24,1,제주특별자치도 서귀포시 남원읍 태위로 647,NaN,http://jjnamwon.jje.es.kr,064-766-4100,064-766-4194,남녀공학,202205,2022-05-31
10850,9290000,제주특별자치도교육청,9299000,제주특별자치도서귀포시교육지원청,50,제주특별자치도,130,서귀포시,보성초등학교,공립,...,19,0,제주특별자치도 서귀포시 대정읍 추사로55번길 6-1,NaN,http://jejuboseong.jje.es.kr,064-797-3300,064-797-3399,남녀공학,202205,2022-05-31


In [9]:
# school 데이터도 위도, 경도 제공하지 않기에 제외
# cctv_cnt = get_node_cnt(gps_comb, cctv[['경도', '위도']])
# school_cnt = get_node_cnt(gps_comb, school[['경도', '위도']])
child_cnt = get_node_cnt(gps_comb, child[['경도', '위도']])
parking1_cnt = get_node_cnt(gps_comb, parking1[['경도', '위도']])
parking2_cnt = get_node_cnt(gps_comb, parking2[['경도', '위도']])
parking_cnt = list(np.array(parking1_cnt) + np.array(parking2_cnt))

In [10]:
# gps_comb['CCTV_cnt'] = cctv_cnt
# gps_comb['school_cnt'] = school_cnt
gps_comb['child_cnt'] = child_cnt
gps_comb['parking_cnt'] = parking_cnt

In [11]:
train = pd.merge(train, gps_comb, how = 'left')
test = pd.merge(test, gps_comb, how = 'left').fillna(0)

#### 2. 제주 공항까지 거리(km)
- train과 test의 시작 위경도 좌표와 제주 공항 위경도 좌표까지의 거리(km)

In [12]:
jeju = (33.506683, 126.493177)

In [13]:
train['j_a_dist'] = [haversine((v[0], v[1]), jeju, unit = 'km') for v in train[['start_latitude', 'start_longitude']].values]
test['j_a_dist'] = [haversine((v[0], v[1]), jeju, unit = 'km') for v in test[['start_latitude', 'start_longitude']].values]

#### 3. 한라산까지 거리(km)
- train과 test의 시작 위경도 좌표와 한라산 위경도 좌표까지의 거리(km)

In [14]:
hanla = 33.36168194, 126.5291548

In [15]:
train['h_a_dist'] = [haversine((v[0], v[1]), hanla, unit = 'km') for v in train[['start_latitude', 'start_longitude']].values]
test['h_a_dist'] = [haversine((v[0], v[1]), hanla, unit = 'km') for v in test[['start_latitude', 'start_longitude']].values]

#### 4. start_node_name과 end_node_name을 key값으로 만들어 LabelEncoding

In [16]:
le = LabelEncoder()

In [17]:
train['node_combination'] = train['start_node_name'] + '_' + train['end_node_name']
test['node_combination'] = test['start_node_name'] + '_' + test['end_node_name']

In [18]:
train['node_combination'] = le.fit_transform(train['node_combination'])

In [19]:
for category in np.unique(test['node_combination']) :
    if category not in le.classes_ :
        le.classes_ = np.append(le.classes_, label)
test['node_combination'] = le.transform(test['node_combination'])

#### 5. 위경도 좌표만으로 Clustering(KMeans)
- Clustering Plotting 결과 군집 수가 6일 때 각 좌표가 명확히 구분되어 6으로 설정

In [20]:
km = KMeans(n_clusters = 6, max_iter = 1000, random_state = 42, n_init = 15)

In [21]:
train['gps_cls'] = km.fit_predict(train[['start_latitude', 'start_longitude', 'end_latitude', 'end_longitude']])
test['gps_cls'] = km.predict(test[['start_latitude', 'start_longitude', 'end_latitude', 'end_longitude']])
# numpy 버전 1.21.4로 다운그레이드하여 문제 해결 ㅜㅜ

#### 6. 공휴일 전후 1 ~ 2일 여부

- 일반적인 공휴일 기준으로 전후 1 ~ 2일을 기간을 더 두어 binary화

In [23]:
train['base_date'] = train['base_date'].astype(str)
test['base_date'] = test['base_date'].astype(str)

In [24]:
train['date'] = train['base_date'].str[4:]
test['date'] = test['base_date'].str[4:]

In [25]:
h_days = ['1231', '0101', '0102', '0129', '0130', '0131', '0201', '0202', '0228', '0229', '0230', '0301', '0302', 
          '0505', '0506', '0507', '0508', '0605', '0607', '0606', '0814', '0815', '0816', '0920', '0921', '0504',
          '0922', '1002', '1003', '1004', '1008', '1009', '1010', '1224', '1225', '1226']

In [26]:
train['in_h_days'] = train['date'].isin(h_days)
test['in_h_days'] = test['date'].isin(h_days)

#### 7. 년도

In [27]:
train['base_date'] = pd.to_datetime(train['base_date'])
test['base_date'] = pd.to_datetime(test['base_date'])

In [28]:
train['year'] = train['base_date'].dt.year
test['year'] = test['base_date'].dt.year

#### 8. 월

In [29]:
train['month'] = train['base_date'].dt.month
test['month'] = test['base_date'].dt.month

#### 9. 최고 제한 속도로 도로 주행시 소요 시간

In [30]:
dist = []
for i, v in enumerate(train[['start_latitude', 'end_latitude', 'start_longitude', 'end_longitude']].values) :
    dist.append(haversine((v[0], v[2]), (v[1], v[3]), unit = 'km'))

In [31]:
train['at_time'] = 60 * pd.Series(dist) / train['maximum_speed_limit']

In [32]:
dist = []
for i, v in enumerate(test[['start_latitude', 'end_latitude', 'start_longitude', 'end_longitude']].values) :
    dist.append(haversine((v[0], v[2]), (v[1], v[3]), unit = 'km'))

In [33]:
test['at_time'] = 60 * pd.Series(dist) / test['maximum_speed_limit']

In [34]:
gc.collect()

110

#### 9. 방위각
- 각 도로의 start, end node의 위경도 좌표로 해당 도로의 방위각 계산

In [35]:
def Azimuth(lat1, lng1, lat2, lng2):
    Lat1 = math.radians(lat1)
    Lat2 = math.radians(lat2)
    Lng1 = math.radians(lng1)
    Lng2 = math.radians(lng2)
    
    y = math.sin(Lng2 - Lng1) * math.cos(Lat2)
    x = math.cos(Lat1) * math.sin(Lat2) - math.sin(Lat1) * math.cos(Lat2) * math.cos(Lng2-Lng1)
    z = math.atan2(y, x)

    a = np.rad2deg(z)
    
    if(a < 0):
        a = 180 + (180 + a)
    return a

In [36]:
train['degree'] = [Azimuth(v[0], v[1], v[2], v[3]) for i, v in enumerate(train[['start_latitude', 'start_longitude', 'end_latitude', 'end_longitude']].values)]
test['degree'] = [Azimuth(v[0], v[1], v[2], v[3]) for i, v in enumerate(test[['start_latitude', 'start_longitude', 'end_latitude', 'end_longitude']].values)]

#### 10. 계절

In [37]:
def get_season(x) :
    
    if x in [9, 10, 11] :
        return 3
    elif x in [12, 1, 2] :
        return 2
    elif x in [3, 4, 5, 6] :
        return 1
    else :
        return 0

In [38]:
train['season'] = train['month'].apply(get_season)
test['season'] = test['month'].apply(get_season)

#### 11. 요일

- 일반적인 요일 순서대로가 아닌 LabelEncoding으로 진행

In [39]:
train['day_of_week'] = le.fit_transform(train['day_of_week'])

In [40]:
for category in np.unique(test['day_of_week']) :
    if category not in le.classes_ :
        le.classes_ = np.append(le.classes_, label)
test['day_of_week'] = le.transform(test['day_of_week'])

#### 12. 도로명

- 도로명 LabelEncoding

In [41]:
train['road_name'] = le.fit_transform(train['road_name'])

In [42]:
for category in np.unique(test['road_name']) :
    if category not in le.classes_ :
        le.classes_ = np.append(le.classes_, label)
test['road_name'] = le.transform(test['road_name'])

#### 13. 시작 노드 == 종료 노드 여부

In [43]:
train['node_same'] = train['start_node_name'] == train['end_node_name']
test['node_same'] = test['start_node_name'] == test['end_node_name']

#### 14. 기타 컬럼 LabelEncoding

In [44]:
train['start_turn_restricted'] = le.fit_transform(train['start_turn_restricted'])

In [45]:
for category in np.unique(test['start_turn_restricted']) :
    if category not in le.classes_ :
        le.classes_ = np.append(le.classes_, label)
test['start_turn_restricted'] = le.transform(test['start_turn_restricted'])

In [46]:
train['end_turn_restricted'] = le.fit_transform(train['end_turn_restricted'])

In [47]:
for category in np.unique(test['end_turn_restricted']) :
    if category not in le.classes_ :
        le.classes_ = np.append(le.classes_, label)
test['end_turn_restricted'] = le.transform(test['end_turn_restricted'])

In [48]:
# 모델링 사용 제외 컬럼 삭제
train.drop(['start_node_name', 'end_node_name', 'date', 'base_date'], axis = 1, inplace = True)
test.drop(['start_node_name', 'end_node_name', 'date', 'base_date'], axis = 1, inplace = True)

### Modeling

- lane_count를 1, 2, 3으로 나누어 모델링
- LGBM, XGBoost는 optuna로 파라미터 튜닝

In [49]:
X = train.drop(['target'], axis = 1)
y = train.target
target = test[X.columns]
skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

X1 = X[X['lane_count'] == 1].drop(['lane_count'], axis = 1)
X2 = X[X['lane_count'] == 2].drop(['lane_count'], axis = 1)
X3 = X[X['lane_count'] == 3].drop(['lane_count'], axis = 1)

y1 = y[X1.index]
y2 = y[X2.index]
y3 = y[X3.index]

standard1 = X1['day_of_week']
standard2 = X2['day_of_week']
standard3 = X3['day_of_week']

target1 = target.loc[target['lane_count'] == 1, X1.columns]
target2 = target.loc[target['lane_count'] == 2, X2.columns]
target3 = target.loc[target['lane_count'] == 3, X3.columns]